Use this notebook to combine data from two source (there is gap in data)

In [4]:
import pandas as pd
import numpy as np
folder_path = "bintulu"

In [ ]:
df1 = pd.read_csv(f"..\data\{folder_path}\{folder_path}-rainfall-daily.csv")
df1["DateTime"] = pd.to_datetime(df1["DateTime"],format="%Y-%m-%d")
df1.set_index("DateTime",inplace=True)

In [5]:
df2 = pd.read_csv(f"..\data\{folder_path}\daily-rainfall-data.csv")
df2["DateTime"] = pd.to_datetime(df2["DateTime"],format="%d/%m/%Y")
# df2.set_index("DateTime",inplace=True)

In [ ]:
# Find common dates between df1 and df2
common_dates = df1.index.intersection(df2['DateTime'])

# Drop the rows from df1 that have DateTime values present in df2
df1 = df1.drop(common_dates)

# Display the updated df1
df1

,Rainfall,TOTAL,ClimAdjust,ANOM,Temperature,DewPoint,Humidity,Visibility,WindSpeed,Pressure,WindDir,Latitude,Longitude,Elevation
DateTime,,,,,,,,,,,,,,
2010-11-26,16.0,25.06,26.70,-1.64,81.090909,74.909091,82.409091,6.000000,5.909091,29.668182,224.000780,4.049213,114.810996,28.7
2010-11-27,16.0,25.06,26.70,-1.64,79.875000,74.416667,84.500000,6.000000,5.750000,29.683750,57.239884,4.049213,114.810996,28.7
2010-11-28,8.0,25.06,26.70,-1.64,80.826087,76.130435,86.217391,6.000000,4.173913,29.671739,27.028382,4.049213,114.810996,28.7
2010-11-29,11.5,25.06,26.70,-1.64,81.125000,76.666667,86.750000,6.000000,4.708333,29.637500,275.746991,4.049213,114.810996,28.7
2010-11-30,17.5,25.06,26.70,-1.64,78.181818,75.545455,91.954545,5.681818,5.590909,29.619091,166.150255,4.049213,114.810996,28.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-27,24.5,27.20,26.72,0.48,81.083333,76.666667,87.250000,5.708333,4.500000,29.680000,207.986653,4.049213,114.810996,28.7
2019-10-28,53.5,27.20,26.72,0.48,79.458333,76.166667,90.125000,5.916667,6.416667,29.711250,197.868289,4.049213,114.810996,28.7
2019-10-29,1.0,27.20,26.72,0.48,80.500000,75.500000,85.250000,6.000000,5.958333,29.716250,224.149031,4.049213,114.810996,28.7


In [100]:
# Check for negative data in the Rainfall column
if (df2['Rainfall'] < 0).any():
    # Forward fill the negative data
    df2['Rainfall'] = df2['Rainfall'].mask(df2['Rainfall'] < 0).ffill()

# Check for negative data in the Rainfall column
print(df1.Rainfall.loc[df1.Rainfall < 0])
print(df2.Rainfall.loc[df2.Rainfall < 0])

Series([], Name: Rainfall, dtype: float64)
Series([], Name: Rainfall, dtype: float64)


In [101]:
oni_data = pd.read_csv("oni.txt", sep="\s+")
oni_data

,YR,MON,TOTAL,ClimAdjust,ANOM
0,1950,1,24.56,26.18,-1.62
1,1950,2,25.07,26.39,-1.32
2,1950,3,25.88,26.95,-1.07
3,1950,4,26.29,27.39,-1.11
4,1950,5,26.19,27.56,-1.37
...,...,...,...,...,...
893,2024,6,27.91,27.73,0.18
894,2024,7,27.34,27.29,0.05
895,2024,8,26.74,26.86,-0.11
896,2024,9,26.47,26.72,-0.25


In [102]:
feature_data = pd.read_csv(f"../data/{folder_path}/rainfall-feature-wunderground.csv")
feature_data.rename(columns={"Time":"DateTime"},inplace=True)
feature_data["DateTime"] = pd.to_datetime(feature_data["DateTime"], format="%Y-%m-%d %H:%M:%S%z")
feature_data["DateTime"] = feature_data["DateTime"].dt.tz_localize(None)
feature_data.set_index("DateTime",inplace=True)
# feature_data.drop(columns={"Wind Gust", "Precip."}, inplace=True)
feature_data.fillna({"Wind Speed":0}, inplace=True)
feature_data.ffill(inplace=True)
feature_data.rename(columns={"Dew Point": "DewPoint", "Wind Speed": "WindSpeed", "Wind Direction": "WindDir"}, inplace=True)
feature_data

,Temperature,DewPoint,Humidity,Visibility,WindSpeed,Pressure,WindDir
DateTime,,,,,,,
2002-08-01 00:00:00,75.0,73.0,94.0,6.0,1.0,29.72,40.0
2002-08-01 01:00:00,75.0,75.0,100.0,6.0,0.0,29.72,40.0
2002-08-01 02:00:00,75.0,73.0,94.0,6.0,3.0,29.69,40.0
2002-08-01 03:00:00,75.0,73.0,94.0,6.0,0.0,29.69,40.0
2002-08-01 04:00:00,73.0,73.0,100.0,6.0,9.0,29.66,40.0
...,...,...,...,...,...,...,...
2024-12-31 19:00:00,84.0,77.0,79.0,6.0,6.0,29.67,270.0
2024-12-31 20:00:00,84.0,77.0,79.0,6.0,6.0,29.70,270.0
2024-12-31 21:00:00,82.0,77.0,84.0,6.0,6.0,29.73,260.0


In [103]:
df2 = pd.merge(
    df2,
    oni_data[["YR", "MON", "TOTAL", "ClimAdjust", "ANOM"]],
    left_on=[df2["DateTime"].dt.year, df2["DateTime"].dt.month],
    right_on= ["YR", "MON"]
).drop(["YR", "MON"], axis=1)
df2.set_index("DateTime",inplace=True)

In [104]:
# Add all columns from df1 to df2, filling with NaN where data is not available
df2 = df2.reindex(columns=df1.columns)

In [105]:
# Update df1 with non-duplicate entries from feature_data

def resultant_wind_direction(x):
    radians = np.deg2rad(x)

    mean_sin = np.mean(np.sin(radians))
    mean_cos = np.mean(np.cos(radians))

    resultant_radians = np.arctan2(mean_sin, mean_cos)
    resultant_degrees = np.rad2deg(resultant_radians) % 360
    
    return resultant_degrees

topo_loc = {
    "lawas":
    {
        "lat": 4.847301, "lon": 115.406703, "elev": 6.5,
    },
    "mulu":
    {
        "lat": 4.049213, "lon": 114.810996, "elev": 28.7,
    },
    "kuching":
    {
        "lat": 1.487123, "lon": 110.341599, "elev": 22.3,
    },
    "kota-kinabalu":
    {
        "lat": 5.923283, "lon": 116.051239, "elev": 5.4,
    },
    "kuantan":
    {
        "lat": 3.780726, "lon": 103.215062, "elev": 16,
    },
    "miri":
    {
        "lat": 4.322991, "lon": 113.987416, "elev": 19,
    }

}

In [106]:
# Create daily aggregations from feature_data and add location data
# Create daily aggregations from feature_data and add location data
daily_data = feature_data.resample('D').agg({
    'Temperature': 'mean',
    'DewPoint': 'mean',
    'Humidity': 'mean',
    'Visibility': 'mean',
    'WindSpeed': 'mean',
    'Pressure': 'mean',
    'WindDir': resultant_wind_direction
})

# Add location information from topo_loc
daily_data['Latitude'] = topo_loc[folder_path]['lat']
daily_data['Longitude'] = topo_loc[folder_path]['lon']
daily_data['Elevation'] = topo_loc[folder_path]['elev']

# Update df2's NaN values with matching data from daily_data
common_dates = df2.index.intersection(daily_data.index)
df2.loc[common_dates, daily_data.columns] = daily_data.loc[common_dates]

In [107]:
# Find common dates between df1 and df2
common_indices = df1.index.intersection(df2.index)

# Update df1 with df2 values where df1 has NaN values
for column in df1.columns:
    if column in df2.columns:
        mask = df1.loc[common_indices, column].isna()
        df1.loc[common_indices[mask], column] = df2.loc[common_indices[mask], column]

In [108]:
df1

,Rainfall,TOTAL,ClimAdjust,ANOM,Temperature,DewPoint,Humidity,Visibility,WindSpeed,Pressure,WindDir,Latitude,Longitude,Elevation
DateTime,,,,,,,,,,,,,,
2010-11-26,16.0,25.06,26.70,-1.64,81.090909,74.909091,82.409091,6.000000,5.909091,29.668182,224.000780,4.049213,114.810996,28.7
2010-11-27,16.0,25.06,26.70,-1.64,79.875000,74.416667,84.500000,6.000000,5.750000,29.683750,57.239884,4.049213,114.810996,28.7
2010-11-28,8.0,25.06,26.70,-1.64,80.826087,76.130435,86.217391,6.000000,4.173913,29.671739,27.028382,4.049213,114.810996,28.7
2010-11-29,11.5,25.06,26.70,-1.64,81.125000,76.666667,86.750000,6.000000,4.708333,29.637500,275.746991,4.049213,114.810996,28.7
2010-11-30,17.5,25.06,26.70,-1.64,78.181818,75.545455,91.954545,5.681818,5.590909,29.619091,166.150255,4.049213,114.810996,28.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-27,24.5,27.20,26.72,0.48,81.083333,76.666667,87.250000,5.708333,4.500000,29.680000,207.986653,4.049213,114.810996,28.7
2019-10-28,53.5,27.20,26.72,0.48,79.458333,76.166667,90.125000,5.916667,6.416667,29.711250,197.868289,4.049213,114.810996,28.7
2019-10-29,1.0,27.20,26.72,0.48,80.500000,75.500000,85.250000,6.000000,5.958333,29.716250,224.149031,4.049213,114.810996,28.7


In [110]:
# Find common dates between df1 and df2
common_dates = df1.index.intersection(df2.index)

print(common_dates)

DatetimeIndex([], dtype='datetime64[ns]', name='DateTime', freq=None)


In [111]:
df1 = pd.concat([df1, df2])
df1

,Rainfall,TOTAL,ClimAdjust,ANOM,Temperature,DewPoint,Humidity,Visibility,WindSpeed,Pressure,WindDir,Latitude,Longitude,Elevation
DateTime,,,,,,,,,,,,,,
2010-11-26,16.0,25.06,26.7,-1.64,81.090909,74.909091,82.409091,6.000000,5.909091,29.668182,224.000780,4.049213,114.810996,28.7
2010-11-27,16.0,25.06,26.7,-1.64,79.875000,74.416667,84.500000,6.000000,5.750000,29.683750,57.239884,4.049213,114.810996,28.7
2010-11-28,8.0,25.06,26.7,-1.64,80.826087,76.130435,86.217391,6.000000,4.173913,29.671739,27.028382,4.049213,114.810996,28.7
2010-11-29,11.5,25.06,26.7,-1.64,81.125000,76.666667,86.750000,6.000000,4.708333,29.637500,275.746991,4.049213,114.810996,28.7
2010-11-30,17.5,25.06,26.7,-1.64,78.181818,75.545455,91.954545,5.681818,5.590909,29.619091,166.150255,4.049213,114.810996,28.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,3.4,27.12,26.6,0.52,80.208333,78.416667,94.375000,5.708333,3.500000,29.756250,16.924704,4.049213,114.810996,28.7
2019-12-28,3.0,27.12,26.6,0.52,79.541667,77.666667,93.791667,5.458333,4.041667,29.748750,59.331488,4.049213,114.810996,28.7
2019-12-29,0.8,27.12,26.6,0.52,83.125000,78.916667,87.041667,6.000000,5.458333,29.740000,46.601550,4.049213,114.810996,28.7


In [112]:
df1.to_csv(f"..\data\{folder_path}\{folder_path}-rainfall-daily.csv")